### Case Study 2
### 🍕 Pizza Runner Part 5

Source : <a href='https://8weeksqlchallenge.com/case-study-2/' style="text-decoration:none">8 Week SQL Challenge</a>
***

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///datasets/pizza_runner.sqlite

<a id='top'></a>

<a id='price'></a>
### <font color='green'> Price and Ratings </font>

#### 1. If a Meat Lovers pizza costs 12 dollar and Vegetarian costs 10 dollar and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

In [3]:
%%sql
DROP TABLE IF EXISTS customer_orders_2;
CREATE TEMP TABLE customer_orders_2 AS
SELECT t1.*
FROM customer_orders t1
WHERE t1.order_id IN (
    SELECT order_id
    FROM runner_orders
    WHERE cancellation = ''
)

 * sqlite:///datasets/pizza_runner.sqlite
Done.
Done.


[]

In [12]:
%%sql
DROP TABLE IF EXISTS meat_veggie_count;
CREATE TEMP TABLE meat_veggie_count AS
SELECT 
    SUM(CASE WHEN pizza_id = 1 THEN 1 ELSE 0 END) AS meatlover_ordered,
    SUM(CASE WHEN pizza_id = 2 THEN 1 ELSE 0 END) AS vegetarian_ordered
FROM customer_orders_2


 * sqlite:///datasets/pizza_runner.sqlite
Done.
Done.


[]

In [13]:
%%sql
SELECT
    meatlover_ordered * 12 + vegetarian_ordered * 10 AS money_made_by_pizza
FROM meat_veggie_count

 * sqlite:///datasets/pizza_runner.sqlite
Done.


money_made_by_pizza
138


#### 2. What if there was an additional 1 dollar charge for any pizza extras?

    Add cheese is $1 extra


In [14]:
%%sql
WITH count_extra_toppings AS(
SELECT 
    SUM(CASE 
    WHEN LENGTH(extras) >1 
    THEN LENGTH(REPLACE(extras,', ',''))
    ELSE LENGTH(extras) END) AS extras
FROM customer_orders_2
)

SELECT
    meatlover_ordered * 12 + vegetarian_ordered * 10 +
    (SELECT extras FROM count_extra_toppings )AS money_made_by_pizza
FROM meat_veggie_count

 * sqlite:///datasets/pizza_runner.sqlite
Done.


money_made_by_pizza
142


### 3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

In [15]:
%%sql
DROP TABLE IF EXISTS ratings;
CREATE TABLE ratings
    (
    order_id INTEGER,
    rating INTEGER
    );
INSERT INTO ratings
    (order_id, rating)
VALUES
    (1,4),
    (2,3),
    (3,5),
    (4,3),
    (5,3),
    (6,2),
    (7,4),
    (8,2),
    (9,3),
    (10,4);

 * sqlite:///datasets/pizza_runner.sqlite
Done.
Done.
10 rows affected.


[]

#### 4.Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?

    customer_id
    order_id
    runner_id
    rating
    order_time
    pickup_time
    Time between order and pickup
    Delivery duration
    Average speed
    Total number of pizzas


In [34]:
%%sql
SELECT t1.customer_id,
       t1.order_id,
       t2.runner_id,
       t3.rating, 
       t1.order_time,
       t2.pickup_time,
       Round((julianday(pickup_time) - julianday(order_time))*24*60) AS time_between_order_pickup ,
       t2.duration AS delivery_duration,
       ROUND(t2.distance / t2.duration ,2) AS avg_speed,
       t1.num_pizza_ordered 
    
FROM(
    SELECT order_id,customer_id,pizza_id,order_time, count(pizza_id) AS num_pizza_ordered
    FROM customer_orders_2
    GROUP BY 1,2,3,4
) t1
JOIN runner_orders t2
ON t1.order_id = t2.order_id
JOIN ratings t3
ON t3.order_id = t1.order_id

 * sqlite:///datasets/pizza_runner.sqlite
Done.


customer_id,order_id,runner_id,rating,order_time,pickup_time,time_between_order_pickup,delivery_duration,avg_speed,num_pizza_ordered
101,1,1,4,2020-01-01 18:05:02,2020-01-01 18:15:34,11.0,32,0.63,1
101,2,1,3,2020-01-01 19:00:52,2020-01-01 19:10:54,10.0,27,0.74,1
102,3,1,5,2020-01-02 23:51:23,2020-01-03 00:12:37,21.0,20,0.67,1
102,3,1,5,2020-01-02 23:51:23,2020-01-03 00:12:37,21.0,20,0.67,1
103,4,2,3,2020-01-04 13:23:46,2020-01-04 13:53:03,29.0,40,0.59,1
103,4,2,3,2020-01-04 13:23:46,2020-01-04 13:53:03,29.0,40,0.59,2
104,5,3,3,2020-01-08 21:00:29,2020-01-08 21:10:57,10.0,15,0.67,1
105,7,2,4,2020-01-08 21:20:29,2020-01-08 21:30:45,10.0,25,1.0,1
102,8,2,2,2020-01-09 23:54:33,2020-01-10 00:15:02,20.0,15,1.56,1
104,10,1,4,2020-01-11 18:34:49,2020-01-11 18:50:20,16.0,10,1.0,2


####  5. If a Meat Lovers pizza was 12 DOLLARS and Vegetarian 10 dollars fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

In [56]:
%%sql
WITH paid_to_runners AS(
    SELECT SUM((distance+0)*0.3) AS paid_to_runners
    FROM runner_orders
),
earned AS(
    SELECT meatlover_ordered *12 + vegetarian_ordered * 10 AS earned
    FROM meat_veggie_count
)

SELECT earned - (SELECT * FROM paid_to_runners) AS leftover_money
FROM earned

 * sqlite:///datasets/pizza_runner.sqlite
Done.


leftover_money
94.44


#### Prev : 🍕 <a href='case_study_2_pizza_part_4.ipynb'> Pizza Runner PART 4 </a>

#### Part 1 : 🍕 <a href='case_study_2_pizza_part_1.ipynb'> Pizza Runner PART 1 </a>

#### Back to : [Top](#top) ▲

<a href='https://github.com/sumedhadewan/8-week-sql-challenge' target='_blank'>See this Project on Github</a>